In [37]:
import json
import math

def axis_angle_to_quaternion(angle_deg, axis):
    """주어진 축(axis: [x,y,z])을 기준으로 angle_deg(도) 회전하는 쿼터니언 [w,x,y,z] 반환"""
    angle_rad = math.radians(angle_deg)
    half = angle_rad / 2.0
    w = math.cos(half)
    sin_half = math.sin(half)
    return [w, axis[0]*sin_half, axis[1]*sin_half, axis[2]*sin_half]

def quaternion_multiply(q1, q2):
    """[w,x,y,z] 형식의 두 쿼터니언을 곱하여 결과 반환"""
    w1, x1, y1, z1 = q1
    w2, x2, y2, z2 = q2
    return [
        w1*w2 - x1*x2 - y1*y2 - z1*z2,
        w1*x2 + x1*w2 + y1*z2 - z1*y2,
        w1*y2 - x1*z2 + y1*w2 + z1*x2,
        w1*z2 + x1*y2 - y1*x2 + z1*w2
    ]

num_frames = 120
cycle = 60  # 60프레임 = 1 보행 사이클
frames = []

for frame in range(num_frames):
    # t: 0 ~ 2 (두 사이클)
    t = frame / cycle
    
    # ------------------------------------------------
    # (1) Spine (Hip ~ Head)
    # Hip은 z축으로 전진 + 약간의 상하 bounce
    hip_y = 0.05 * math.sin(2 * math.pi * t)   # 상하 bounce
    hip_z = 0.1 * frame                       # 전진
    hip = [0.0, hip_y, hip_z]
    
    # 어깨가 머리보다 높이 있지 않도록, Spine ~ Head까지 조금 길게 잡음
    # 예) Hip(0,?), Spine(1.2), Chest(0.6), Neck(0.4), Head(0.5)
    spine_pos = [
        hip,               # 0: Hip (Root)
        [0, 0.5, 0],       # 1: Spine
        [0, 1.5, 0],       # 2: Chest
        [0, 1, 0],       # 3: Neck
        [0, 0.5, 0]        # 4: Head
    ]
    
    # Hip에만 약간 yaw 회전
    hip_rot = axis_angle_to_quaternion(5 * math.sin(2*math.pi * t), [0,1,0])
    spine_rot = [
        hip_rot,           # Hip
        [1,0,0,0],         # Spine
        [1,0,0,0],         # Chest
        [1,0,0,0],         # Neck
        [1,0,0,0]          # Head
    ]
    
    # ------------------------------------------------
    # (2) Upper Body (Arms)
    # 어깨가 머리보다 낮게 위치하도록, y=1.2 정도로 맞춤
    # 또한 팔 관절 길이를 조금 더 늘려서 붙어 보이지 않게 조정
    left_arm_pos = [
        [-1, -1.2, 0.0],  # (0) 어깨
        [0.0, -0.5, 0.0],  # (1) 상완
        [0.0, -0.5, 0.0],  # (2) 팔꿈치
        [0.0, -0.3, 0.0],  # (3) 손목
        [0.0, -0.2, 0.0]   # (4) 손
    ]
    right_arm_pos = [
        [1, -1.2, 0.0],   # (0) 어깨
        [0.0, -0.5, 0.0],  # (1) 상완
        [0.0, -0.5, 0.0],  # (2) 팔꿈치
        [0.0, -0.3, 0.0],  # (3) 손목
        [0.0, -0.2, 0.0]   # (4) 손
    ]
    
    # 팔 회전: 기본 보행처럼, 어깨와 팔꿈치에만 x축 회전
    angle_shoulder_left = 20 * math.sin(2 * math.pi * t)
    angle_elbow_left = 10 * math.sin(4 * math.pi * t)
    q_shoulder_left = axis_angle_to_quaternion(angle_shoulder_left, [1,0,0])
    q_elbow_left = axis_angle_to_quaternion(angle_elbow_left, [1,0,0])
    
    left_arm_rot = [
        q_shoulder_left,   # 어깨
        [1,0,0,0],         # 상완
        q_elbow_left,      # 팔꿈치
        [1,0,0,0],         # 손목
        [1,0,0,0]          # 손
    ]
    
    angle_shoulder_right = 20 * math.sin(2 * math.pi * t + math.pi)
    angle_elbow_right = 10 * math.sin(4 * math.pi * t + math.pi)
    q_shoulder_right = axis_angle_to_quaternion(angle_shoulder_right, [1,0,0])
    q_elbow_right = axis_angle_to_quaternion(angle_elbow_right, [1,0,0])
    
    right_arm_rot = [
        q_shoulder_right,
        [1,0,0,0],
        q_elbow_right,
        [1,0,0,0],
        [1,0,0,0]
    ]
    
    # ------------------------------------------------
    # (3) Lower Body (Legs)
    # 다리도 관절 길이를 조금 더 늘림
    left_leg_pos = [
        [-0.3, 0, 0],      # (0) Left Hip
        [0, -0.6, 0],      # (1) Thigh
        [0, -0.6, 0],      # (2) Knee
        [0, -0.6, 0],      # (3) Shin
        [0, -0.3, 0],      # (4) Ankle
        [0, -0.2, 0.2],    # (5) Foot
        [0, 0, 0.1]        # (6) Toes
    ]
    right_leg_pos = [
        [0.3, 0, 0],       # (0) Right Hip
        [0, -0.6, 0],      # (1) Thigh
        [0, -0.6, 0],      # (2) Knee
        [0, -0.6, 0],      # (3) Shin
        [0, -0.3, 0],      # (4) Ankle
        [0, -0.2, 0.2],    # (5) Foot
        [0, 0, 0.1]        # (6) Toes
    ]
    
    # 다리 회전: Hip(첫 관절)과 무릎(두 번째 관절)만 단순 x축 회전
    angle_left_leg = 20 * math.sin(2 * math.pi * t + math.pi)
    angle_right_leg = 20 * math.sin(2 * math.pi * t)
    q_left_leg = axis_angle_to_quaternion(angle_left_leg, [1,0,0])
    q_right_leg = axis_angle_to_quaternion(angle_right_leg, [1,0,0])
    
    knee_left = 10 * math.sin(4 * math.pi * t + math.pi)
    knee_right = 10 * math.sin(4 * math.pi * t)
    q_knee_left = axis_angle_to_quaternion(knee_left, [1,0,0])
    q_knee_right = axis_angle_to_quaternion(knee_right, [1,0,0])
    
    left_leg_rot = [
        q_left_leg,  # Hip
        [1,0,0,0],   # Thigh
        q_knee_left, # Knee
        [1,0,0,0],   # Shin
        [1,0,0,0],   # Ankle
        [1,0,0,0],   # Foot
        [1,0,0,0]    # Toes
    ]
    right_leg_rot = [
        q_right_leg,
        [1,0,0,0],
        q_knee_right,
        [1,0,0,0],
        [1,0,0,0],
        [1,0,0,0],
        [1,0,0,0]
    ]
    
    # ------------------------------------------------
    frame_data = {
        "spine": {
            "position": spine_pos,
            "rotation": spine_rot
        },
        "upper_body": {
            "left": {
                "position": left_arm_pos,
                "rotation": left_arm_rot
            },
            "right": {
                "position": right_arm_pos,
                "rotation": right_arm_rot
            }
        },
        "lower_body": {
            "left": {
                "position": left_leg_pos,
                "rotation": left_leg_rot
            },
            "right": {
                "position": right_leg_pos,
                "rotation": right_leg_rot
            }
        }
    }
    frames.append(frame_data)

data = {"frames": frames}

with open("walk_animation.json", "w") as f:
    json.dump(data, f, indent=4)
